# 3. Julia的函数
## 3.1 定义函数
定义函数的常规语法如下
```[Julia]
function fname(arglist)
# function body ...
return value(s)
```
函数的变量清单也可以是空的，则被写作 fname()。
例如

In [8]:
function mult(x, y)
    println("x is $x and y is $y")
    return x * y
end

mult (generic function with 1 method)

调用函数，有

In [7]:
mult(3,5)
n = mult(3,4)

LoadError: UndefVarError: mult not defined

通常按照惯例，无特殊强调时，函数名用小写字母表示。可以包含unicode字符，对于数学符号比较有用。

return关键字是可选的，也可以直接写成 x * y 则函数默认返回end前的最后一行。多行函数中，写return能够提高程序的可读性。

return不仅能输出一个结果变量，也能输出多个。
例如

In [6]:
function multi(n,m)
    n*m, div(n,m), n%m
end

multi (generic function with 1 method)

In [11]:
multi(8, 2)

(16, 4, 0)

也可以将输出分别赋值给不同的变量。

In [13]:
x, y, z = multi(8, 2)
x

16

也可以用varargs 来定义函数，比如

In [14]:
function varargs(n, m, args...)
    println("arguments: $n, $m, $args")
end

varargs (generic function with 1 method)

此时，n和m是位置变量，可以不给或者更多。args... 语句表示剩下的所有参数。比如

In [15]:
varargs(1,2,3,4)

arguments: 1, 2, (3, 4)


或者

In [21]:
varargs(1,3)

arguments: 1, 3, ()


Julia中，函数的所有参数都通过引用传递，在传递过程中它们的数值不会被复制，这意味着它们可以从内部进行更改，并且更改将对代码调用可见。

如下

In [24]:
function insert_elem(arr)
    push!(arr, -10)
end
arr = [2, 3, 4]
insert_elem(arr)
arr

4-element Vector{Int64}:
   2
   3
   4
 -10

可见调用函数过程中，arr的数值已经被更改了。

此外可以显式指定参数的类型，比如

In [ ]:
function multf(x::Float64, y::Float64)
    x * y
end
multf(5, 6)

上述程序输出结果为
```
MethodError: no method matching multf(::Int64, ::Int64)
```
对于简单的数学表达，函数可以写成紧凑的形式，比如

In [28]:
f(x, y) = x^3 - y + x * y
f(3,2)

31

## 3.2 可选参数和关键字参数
定义函数时，一个或者多个参数可以用以下形式赋默认值
```[Julia]
f(arg = val)
```
此时，如果调用时未给出参数值，则自动使用默认值。

In [29]:
f5(a, b = 5) = a + b
f5(1)

6

In [30]:
f5(2,1)

3

上面定义的函数，参数只能根据位置定义，而调用
```[Julia]
f(2, b = 5)
```
则会报错。为了提高代码可读性，在调用函数时显式调用参数值，这种方式称为可选关键字参数，因为参数有明确的名称，因此顺序是无所谓的，但是：
1. 要放到后面
2. 定义时与其它参数用分号隔开

In [35]:
k(x; a1 = 1, a2 = 2) = x * (a1 + a2)
k(3, a2 = 3)

12

举一反三以下，这里面的跟前面混淆，定义时，分号隔开就行。不用赋默认值也可以。

In [34]:
k(x; a1, a2) = x * (a1 - a2)
k(3, a1 = 2, a2 = 1)

3

正常参数，可选位置参数以及关键字参数可以结合使用，比如

In [36]:
function allargs(a, b = 2; c = 3)
    a + b + c
end

allargs (generic function with 2 methods)

In [42]:
println(allargs(1))
println(allargs(1, 3))
# println(allargs(1, 4, 5)) 这个不对，无法输出
println(allargs(1, 4, c = 4))
println(allargs(1, c = 5))

6
7
9
8


一种有用的情况如下

In [50]:
function varargs2(;args...)
    args
end
d=varargs2(k1 = "name1", l2 = "name2",c = 7)

pairs(::NamedTuple) with 3 entries:
  :k1 => "name1"
  :l2 => "name2"
  :c  => 7

In [52]:
d[:k1]

"name1"

## 3.3 匿名函数
函数可以写作没有函数名的形式，如
```[Julia]
(x, y) -> x^3 - y + x * y
```
并且可以通过如下形式为函数绑定一个函数名，并进行调用。

In [1]:
f = (x, y) -> x^3 - y + x * y
f(3, 2)

31

匿名函数也经常写作如下形式：

In [3]:
function (x)
    x + 2
end

#5 (generic function with 1 method)

In [4]:
ans(3)

5

此外，匿名函数通常可以写作lamba表达式的形式，如
```[Julia]
(x) -> x + 2
```
符号“->”前的部分是函数的参数，之后的是返回值。也可以简写作
```[Julia]
x -> x + 2
```
无参数的函数可以写作
```[Julia]
() -> println("hello")
```
**如果在意性能，尽量使用非匿名函数，匿名函数通常用于将一个函数作为参数传递给其它函数时。**

## 3.4 一类函数和闭包
本节讨论函数的功能和灵活性，首先，函数也有自己的类型，并且函数可以幅值给一个变量。

In [9]:
m = mult 
m(6, 6)

x is 6 and y is 6


36

这个功能在使用匿名函数时比较方便，比如

In [11]:
plustwo = function(x)
x + 2
end
plustwo(3)

5

函数可以调用其它函数作为参数，比如

In [12]:
function numerical_derivative(f, x, dx=0.01)
    derivative = (f(x+dx) - f(x-dx))/(2*dx)
    return derivative
end
f = x -> 2x^2 + 30x + 9
println(numerical_derivative(f, 1, 0.001))

33.99999999999537


函数还可以返回另一个函数作为其返回值，例如

In [17]:
function derivative(f)
    return function(x)
        # pick a small value for h
        h = x == 0 ? sqrt(eps(Float64)) : sqrt(eps(Float64)) * x
        xph = x + h
        dx = xph - x
        f1 = f(xph)
        f0 = f(x)
        return(f1 - f0) / dx
    end
end
df =derivative(f)
df(1.0)

34.0

上面两个例子都是匿名函数的应用。

下面是一个计数函数，该函数返回两个匿名函数。

In [18]:
function counter()
    n = 0
    () -> n += 1, () -> n = 0
end

counter (generic function with 1 method)

调用是将两个函数赋值给不同变量

In [19]:
(addOne, reset) = counter()

(var"#23#25"(Core.Box(0)), var"#24#26"(Core.Box(0)))

In [20]:
addOne()

1

In [21]:
addOne()

2

In [22]:
addOne()

3

In [23]:
reset()

0

In [24]:
addOne()

1

可以看到，变量n仅能通过函数addOne和reset来调用，因此这两个函数对于n是封闭的，因此称作闭包？(closures).
局部套用，